In [4]:
import tensorflow as tf
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split

# unpack the pickle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
dataset_dir = '/content/drive/MyDrive/FALL_2022_SEMESTER/COMS_4995_Applied_Deep_Learning/Course_Project/music_dataset/'
pickle_file_path = os.path.join(dataset_dir, 'mel_fcc_pic.pickle')

In [6]:
# unpack the pickle file
pickle_in = open(pickle_file_path, "rb")
test_data = pickle.load(pickle_in)

In [7]:
# map each raga str to an int
raga_to_int = {
    'Sankarabaranam': 0,
    'Reethigowla': 1,
    'Kalyani': 2,
    'Shanmukhapriya': 3,
    'SindhuBhairavi': 4,
    'Sri': 5,
    'Surutti': 6,
    'Thodi': 7,
    'PoorviKalyani': 8,
    'Mohanam': 9,
    'Madhyamavathi': 10,
    'Kharaharapriya': 11,
    'Keeravani': 12,
    'Kapi': 13,
    'Kambhodhi': 14,
    'Kamas': 15, 
    'Jaganmohini': 16,
    'Hamsadhwani': 17,
    'Bilahari': 18,
    'Bhairavi': 19
}


In [8]:
# OK so now just create a random network and see how it does
# split up into test and train data
X = []
y = []
for val in test_data:
  X.append(val[0])
  y.append(raga_to_int[val[1]])

del test_data




In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)
X_train =np.array(X_train)
X_test =np.array(X_test)
y_train =np.array(y_train)
y_test =np.array(y_test)

In [10]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1173, 20, 2584)
X_test shape: (391, 20, 2584)
y_train shape: (1173,)
y_test shape: (391,)


In [19]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(20, 2584)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(20)
])

In [20]:
predictions = model(X_train[:1]).numpy()
predictions

array([[-386.20383  ,  -81.484    ,   70.528946 ,   86.21089  ,
         255.5815   , -197.7587   , -129.8013   ,  114.34379  ,
         264.70117  ,  -95.483734 ,  -23.135271 ,  526.65137  ,
         127.84048  ,  -25.533188 ,   -3.8497696,    2.7688408,
         -52.48697  ,  342.6666   ,  147.76755  , -162.94676  ]],
      dtype=float32)

In [21]:
tf.nn.softmax(predictions).numpy()


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [22]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [23]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=30)


Epoch 1/30
37/37 [==============================] - 2s 46ms/step - loss: 980.5435 - accuracy: 0.1432
Epoch 2/30
37/37 [==============================] - 2s 49ms/step - loss: 2.9786 - accuracy: 0.1355
Epoch 3/30
37/37 [==============================] - 2s 53ms/step - loss: 2.9599 - accuracy: 0.1355
Epoch 4/30
37/37 [==============================] - 2s 44ms/step - loss: 2.9412 - accuracy: 0.1355
Epoch 5/30
37/37 [==============================] - 2s 45ms/step - loss: 2.9230 - accuracy: 0.2217
Epoch 6/30
37/37 [==============================] - 2s 45ms/step - loss: 2.9054 - accuracy: 0.2285
Epoch 7/30
37/37 [==============================] - 2s 44ms/step - loss: 2.8881 - accuracy: 0.2285
Epoch 8/30
37/37 [==============================] - 2s 43ms/step - loss: 2.8715 - accuracy: 0.2285
Epoch 9/30
37/37 [==============================] - 2s 60ms/step - loss: 2.8555 - accuracy: 0.2285
Epoch 10/30
37/37 [==============================] - 2s 51ms/step - loss: 2.8402 - accuracy: 0.2285
Epoch 1

In [25]:
# might have started to overfit on trarin accuracy after 1 epoch???
# try testing it 
model.evaluate(X_test,  y_test, verbose=2)


13/13 - 0s - loss: 2.6893 - accuracy: 0.2097 - 304ms/epoch - 23ms/step


[2.6892848014831543, 0.20971867442131042]

In [ ]:
# ok so now, get the actual features of the dataset (Spectral centroid, spectral bandwidth, mel-spectrogram, Mel freq cepstral coeff, chroma)
# Get those into a pickle file, and see how it performs on the same neural network. If accuracy is low, then either modify network or add more data samples